<a href="https://colab.research.google.com/github/teploe-odealko/KazanExpress_Junior_DS_test/blob/master/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import joblib
import nltk
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
import pandas as pd
import numpy as np

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
!gdown https://drive.google.com/uc?id=1E7wnU9sT3WmrczdpxfwdzlZiLyZjHOX1


Downloading...
From: https://drive.google.com/uc?id=1E7wnU9sT3WmrczdpxfwdzlZiLyZjHOX1
To: /content/model.joblib
319MB [00:01, 170MB/s]


#Models

In [4]:
input_size = 100

In [5]:
!pip install pytorch_lightning

     |████████████████████████████████| 922 kB 8.1 MB/s 
     |████████████████████████████████| 636 kB 46.2 MB/s 
     |████████████████████████████████| 829 kB 45.3 MB/s 
     |████████████████████████████████| 119 kB 39.9 MB/s 
     |████████████████████████████████| 282 kB 44.8 MB/s 
     |████████████████████████████████| 1.3 MB 27.5 MB/s 
     |████████████████████████████████| 142 kB 62.6 MB/s 
     |████████████████████████████████| 294 kB 72.0 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=8a5d590518607110b47f3bd7127e922a6e7ff6b7fab9e552401e1c05a798b03f
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: future
    Found existing installation: future 0.16.0
    Uninstalling future-0

In [6]:
import torch 
  
from torch import nn 
import pytorch_lightning as pl 
import torch.nn.functional as F 
from torchvision import datasets, transforms
from torch.optim import SGD 
from torch.utils.data import random_split, DataLoader 
from sklearn.metrics import accuracy_score
pl.utilities.seed.seed_everything(seed=42)

class ModelLayer2(pl.LightningModule): 
    def __init__(self): 
        super().__init__() 

        self.fc1 = nn.Linear(100, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 128) 
        self.fc3 = nn.Linear(128, 3) 
        # self.out = nn.Linear(128, len(np.unique(y))) 
        self.lr = 0.01
        self.loss = nn.CrossEntropyLoss()
    
    def forward(self, x):
        batch_size, _, = x.size() 
        x = x.view(batch_size, -1) 
        x = F.relu(self.fc1(x))
        x = self.bn1(x)
        x = F.relu(self.fc2(x)) 
        return self.fc3(x) 
    
    def predict_step(self, batch, batch_idx: int , dataloader_idx: int = None):
        self.eval()
        return self(batch)

    def configure_optimizers(self):
        return torch.optim.SGD(self.parameters(), lr = self.lr) 
    
    def training_step(self, train_batch, batch_idx): 
        x, y = train_batch 
        logits = self.forward(x) 
        loss = self.loss(logits, y) 
        return loss 
    
    def validation_step(self, valid_batch, batch_idx): 
        x, y = valid_batch 
        logits = self.forward(x)
        loss = self.loss(logits, y)

        return {'pred': logits,'target': y}

    def validation_epoch_end(self, outputs):
        y = torch.cat([out['target'] for out in outputs])
        y_hat = torch.cat([out['pred'] for out in outputs])
        prec, recall, f1_macro, _ = precision_recall_fscore_support(
            y.cpu(), y_hat.argmax(axis=1).cpu(), average='macro'
            )
        prec_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(
            y.cpu(), y_hat.argmax(axis=1).cpu(), average='weighted'
            )
        
        acc = accuracy_score(y, y_hat.argmax(axis=1).cpu())
        self.log('val_prec', prec, prog_bar=True)
        self.log('val_recall', recall, prog_bar=True)
        self.log('val_f1_macro', f1_macro, prog_bar=True)
        self.log('val_f1_weighted', f1_weighted, prog_bar=True)
        self.log('val_acc', acc, prog_bar=True)

    def test_step(self, valid_batch, batch_idx): 
        x, y = valid_batch 
        logits = self.forward(x)
        loss = self.loss(logits, y)

        return {'pred': logits,'target': y}

    def test_epoch_end(self, outputs):
        y = torch.cat([out['target'] for out in outputs])
        y_hat = torch.cat([out['pred'] for out in outputs])
        prec, recall, f1_macro, _ = precision_recall_fscore_support(
            y.cpu(), y_hat.argmax(axis=1).cpu(), average='macro'
            )
        prec_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(
            y.cpu(), y_hat.argmax(axis=1).cpu(), average='weighted'
            )
        
        acc = accuracy_score(y, y_hat.argmax(axis=1).cpu())
        self.log('test_prec', prec, prog_bar=True)
        self.log('test_recall', recall, prog_bar=True)
        self.log('test_f1_macro', f1_macro, prog_bar=True)
        self.log('test_f1_weighted', f1_weighted, prog_bar=True)
        self.log('test_acc', acc, prog_bar=True)
  
class ProductsCategoryDataModule(pl.LightningDataModule):
    def __init__(self, X, y, y_strat):
        super().__init__()

        self.X = X
        self.y = y
        self.y_strat = y_strat
        self.batch_size = 32
        self.transform = transforms.Compose([
            transforms.ToTensor()
        ])
  
    def prepare_data(self):
        pass
        
    def setup(self, stage=None):
        self.train_data, self.valid_data, self.test_data = make_datasets(self.X, self.y, self.y_strat)

  
    def train_dataloader(self):
                return DataLoader(self.train_data, 
                          batch_size = self.batch_size)
  
    def val_dataloader(self):
                return DataLoader(self.valid_data,
                          batch_size = self.batch_size)
  
    def test_dataloader(self):
                return DataLoader(self.test_data,
                          batch_size = self.batch_size)

Global seed set to 42


In [7]:
class ModelLayer3(ModelLayer2): 
    def __init__(self): 
        super().__init__()
        # self.fc1.freeze() 
        # self.fc2.freeze() 
        # self.fc3.freeze() 
        self.fc4 = nn.Linear(3, 64)
        self.fc5 = nn.Linear(64, 256)
        self.bn2 = nn.BatchNorm1d(256)
        self.fc6 = nn.Linear(256, 128)
        self.fc7 = nn.Linear(128, 36)

    def forward(self, x):
        batch_size, _, = x.size() 
        x = x.view(batch_size, -1) 
        x = F.relu(self.fc1(x)) 
        x = F.relu(self.fc2(x)) 
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.bn2(x)
        x = F.relu(self.fc6(x))
        return self.fc7(x)

class ModelLayer4(ModelLayer3): 
    def __init__(self): 
        super().__init__()

        self.fc8 = nn.Linear(36, 64)
        self.fc9 = nn.Linear(64, 256)
        self.dr1 = nn.Dropout(0.5)
        self.fc10 = nn.Linear(256, 211)

    def forward(self, x):
        batch_size, _, = x.size() 
        x = x.view(batch_size, -1) 
        x = F.relu(self.fc1(x)) 
        x = F.relu(self.fc2(x)) 
        x = F.relu(self.fc3(x)) 
        x = F.relu(self.fc4(x)) 
        x = F.relu(self.fc5(x)) 
        x = F.relu(self.fc6(x)) 
        x = F.relu(self.fc7(x)) 
        x = F.relu(self.fc8(x)) 
        x = F.relu(self.fc9(x))
        # x = self.dr1(x)
        return self.fc10(x)


class ModelLayer5(ModelLayer4): 
    def __init__(self): 
        super().__init__()

        self.fc11 = nn.Linear(211, 256)
        self.bn3 = nn.BatchNorm1d(256)
        self.fc12 = nn.Linear(256, 512)
        self.fc13 = nn.Linear(512, 481)

    def forward(self, x):
        batch_size, _, = x.size() 
        x = x.view(batch_size, -1) 
        x = F.relu(self.fc1(x)) 
        x = self.bn1(x)
        x = F.relu(self.fc2(x)) 
        x = F.relu(self.fc3(x)) 
        x = F.relu(self.fc4(x)) 
        x = F.relu(self.fc5(x)) 
        x = self.bn2(x)
        x = F.relu(self.fc6(x)) 
        x = F.relu(self.fc7(x)) 
        x = F.relu(self.fc8(x)) 
        x = F.relu(self.fc9(x))
        x = F.relu(self.fc10(x))
        x = F.relu(self.fc11(x))
        x = self.bn3(x)
        x = F.relu(self.fc12(x))
        # x = self.dr1(x)
        return self.fc13(x)

In [8]:
class NNModel():
    def __init__(self,
                 model_level_4,
                 model_level_5,
                 mapping_level_4,
                 mapping_level_5,
                 category_name_df):
        self.model_level_4 = model_level_4
        self.model_level_5 = model_level_5
        self.mapping_level_4 = mapping_level_4
        self.mapping_level_5 = mapping_level_5
        self.cat = category_name_df
    def fit():
        pass
    def predict(self, x):
        x = torch.Tensor(x).reshape(1, -1)

        self.model_level_5.eval()
        y_hat = int(self.model_level_5(x)[0].argmax().numpy())
        category_id = self.mapping_level_5[y_hat]
        if category_id == 0:
            y_hat = int(self.model_level_4(x)[0].argmax().numpy())
            category_id = self.mapping_level_4[y_hat]

        category_5_level = self.cat[self.cat['category_5_level'] == category_id]
        if (len(category_5_level) != 0):
            category_name = category_5_level.category_title.iloc[0]
        else:
            category_name = self.cat[self.cat['category_4_level'] == category_id].category_title.iloc[0]

        return {'category_id' : category_id, 'category_name': category_name}


In [9]:
class Vectorizer():
    def __init__(self, w2v_model):
        self.w2v_model = w2v_model
    def preprocess_text(self, text):
        """Preprocess text into normalised tokens."""
        stop_word_list = nltk.corpus.stopwords.words('russian')
        tokeniser = RegexpTokenizer("[A-Za-zА-Яа-я]+")
        tokens = tokeniser.tokenize(text)
        
        # Lowercase and lemmatise
        # morph = pymorphy2.MorphAnalyzer()
        # tokens_norm = [morph.parse(t.lower())[0].normal_form for t in tokens]
        # tokens_clean = [t for t in tokens_norm if t not in stop_word_list]
        tokens_lower = [t.lower() for t in tokens]
        tokens_clean = [t for t in tokens_lower if t not in stop_word_list]
        return ' '.join(tokens_clean)

    def vectorize(self, text):
        word_embs = [self.w2v_model.wv[w] for w in text.split() if w in self.w2v_model.wv]
        if (len(word_embs) > 0):
            return np.mean(word_embs, axis=0)
        return np.zeros(input_size)


    def fit():
        pass
        
    def transform(self, x):
        preprocessed = self.preprocess_text(x)
        # print(self.w2v_model.wv[preprocessed])
        features_x = self.vectorize(preprocessed)
        return features_x

#Demo

In [10]:
pipeline = joblib.load('/content/model.joblib')


In [12]:
print(pipeline.predict('Массажер для ног'))

{'category_id': 11027, 'category_name': 'Массажеры для лица и тела'}


In [ ]:
print(pipeline.predict('Зарядка для телефона'))
print(pipeline.predict('Пластиковая емкость для хранения'))
print(pipeline.predict('Пластиковый контейнер'))

{'category_id': 12171, 'category_name': 'Кабели'}
{'category_id': 13674, 'category_name': 'Контейнеры и ланч-боксы'}
{'category_id': 13674, 'category_name': 'Контейнеры и ланч-боксы'}


In [ ]:
print(pipeline.predict('Наушники'))
print(pipeline.predict('Наушники для телефона'))

{'category_id': 12980, 'category_name': 'Беспроводные наушники'}
{'category_id': 2803, 'category_name': 'Проводные наушники'}
